In [1]:
import sys
sys.path.append('..')

from word_analogy_tool.web.embeddings import fetch_SG_GoogleNews, fetch_GloVe, fetch_FastText

model = fetch_GloVe(corpus="wiki-6B", dim=300)
# model = fetch_SG_GoogleNews(normalize=False, lower=False, clean_words=False)
# model = fetch_FastText(lang="en", normalize=False, lower=False, clean_words=False)
dict_low = True

print('... model loaded ...')

File already downloaded, skipping
... model loaded ...


In [2]:
from utils.corpuscont import retrieve_corpus

# load corpus 
path1 = r"/home/hanselowski/jupiter_projects/vecSpace_decomp/datasets/googe_cat_corpus.txt"
path2 = r"/home/hanselowski/jupiter_projects/vecSpace_decomp/datasets/wordnet_cat_corpus.txt"
path3 = r"/home/hanselowski/jupiter_projects/vecSpace_decomp/datasets/closed_cat_corpus.txt"

googe_cat_corpus = retrieve_corpus(path1, dict_low)
wordnet_cat_corpus = retrieve_corpus(path2, dict_low)
closed_cat_corpus = retrieve_corpus(path3, dict_low)
corp = wordnet_cat_corpus

In [7]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, recall_score, precision_score
import random

def trainData(single_class, pp):
    # extract training data
    X_in, Y_in = [], []
    sum_examples = 0
    # print('')
    examples = int(round(len(single_class)*pp))
    # print('number of examples: '+str(examples))
    # print(ss)
    sum_examples += examples
    for i in range(examples):
        X_in.append(model[single_class[i]])
        Y_in.append(1)
    return X_in, Y_in, sum_examples        

def testData(single_class, pp):
    # prepare testing data   
    X_t, Y_t = [], []
    examples = int(round(len(single_class)*pp))
    for i in range(examples, len(single_class)):
        X_t.append(model[single_class[i]])
        Y_t.append(1)
    return X_t, Y_t              
            
def nagitveClassData(model, add_data, single_class):
    words = model.vocabulary.words
    dic = [words[i] for i in range(len(words))]
    # dic = [words[i] for i in range(len(add_data))]
    random.shuffle(dic)
    X_a, Y_a = [], []
    cnt = 0
    while add_data > cnt:
        incldue_word = True
        if dic[i] in single_class:
            incldue_word = False
            break
        if incldue_word:
            # print(dic[cnt])
            X_a.append(model[dic[cnt]])
            Y_a.append(0)
            cnt += 1
    return X_a, Y_a           
                
def printScores(Y_true, Y_pred):
    target_names = []
    for i in range(0, 2):
        target_names.append('class '+str(i))
    print(classification_report(    Y_true, Y_pred, target_names = target_names))
    print('precision_score: '+str(precision_score(Y_true, Y_pred, average='macro')))
    print('recall_score: '+   str(   recall_score(Y_true, Y_pred, average='macro')))
    print('f1 macro: '+       str(       f1_score(Y_true, Y_pred, average='macro')))

def evaluate(Y_true, Y_pred):
    inst_sum = Y_true.count(1)
    cor_pred, sum_pred = 0.0, 0.0
    for t, p in zip(Y_true, Y_pred):
        if p == 1:
            sum_pred += 1.0
            if t == 1:
                cor_pred += 1.0
    prec  =  cor_pred/sum_pred           
    recall = cor_pred/inst_sum 
    f1 = 2*prec*recall/(prec+recall)
    '''
    print('')
    print('prec  : ', str(prec))
    print('recall: ', str(recall))
    print('f1    : ', str(f1))
    '''
    return prec, recall, f1
    
# parameters
itter = 5  
add_data = 50000 
# add_data = 500
# add_data = 1000
num_classes = len(corp) 

inst_pp = [0.1, 0.2, 0.3, 0.4, 0.5]
# inst_pp = [0.3]
for pp in inst_pp:
    # j = 0
    f1_overall, recall_overall, precision_overall = 0.0, 0.0, 0.0
    for j in range(num_classes): 
        X_in, Y_in, sum_examples = trainData(corp[j], pp)
        X_t, Y_t   = testData(corp[j], pp)   
        # print('Y_in',str(Y_in))
        # print('Y_t',str(Y_t))

        X, Y = [], []
        f1_scrore_average, recall_scrore_average, precision_scrore_average = 0.0, 0.0, 0.0 
        for i in range(itter):

            X_a, Y_a   = nagitveClassData(model, add_data, corp[j])
            print('num of neg examples: ', len(Y_a))
            # negative samples and samples for prediction
            sum_examples = 500
            X_a_r, X_a_t = X_a[:sum_examples], X_a[sum_examples:]
            Y_a_r, Y_a_t = Y_a[:sum_examples], Y_a[sum_examples:]
            # print('Y_a_r',str(Y_a_r))
            # print('Y_a_t',str(Y_a_t))

            # randomize training examples    
            combined = list(zip(X_in+X_a_r, Y_in+Y_a_r))
            random.shuffle(combined)
            X[:], Y[:] = zip(*combined)
            # print('sum of examples: '+str(sum_examples))
            # print('len lables: '+str(len(Y)))

            clf = svm.SVC(kernel='linear', class_weight='balanced') 
            clf.fit(X, Y) 
            # print('fit Y',str(Y))

            Y_pred = clf.predict(X+X_t+X_a_t)
            Y_true = Y+Y_t+Y_a_t
            # print('Y_pred',str(Y_pred))
            # print('Y_true = Y_in+Y_a_r+Y_t+Y_a_t',str(Y_true))
            # printScores(Y_true, Y_pred)
            
            prec, recall, f1 = evaluate(Y_true, Y_pred)
            
            f1_scrore_average += f1
            recall_scrore_average += recall
            precision_scrore_average += prec
        '''
        print('')
        print('precision_scrore_average: '+ str(precision_scrore_average/itter))
        print('recall_scrore_average: '+ str(recall_scrore_average/itter))
        print('f1_score_average: '+ str(f1_scrore_average/itter))
        '''
        precision_overall += precision_scrore_average/itter
        recall_overall += recall_scrore_average/itter
        f1_overall += f1_scrore_average/itter
        
        
    print('')
    print('pp: ', pp)
    print('precicion_overall: '+ str(precision_overall/num_classes))
    print('recall_overall: '+ str(recall_overall/num_classes))
    print('f1 overall: '+ str(round(f1_overall/num_classes, 3)))
    print('')    

    

num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
n

num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000

pp:  0.3
precicion_overall: 0.9688472667928661
recall_overall: 0.6847614452888667
f1 overall: 0.793

num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of neg examples:  1000
num of 

In [4]:
#######################
# wordnet corpus new ------------------

add_data = 50000












add_data = 500

pp:  0.1
precicion_overall: 1.0
recall_overall: 0.22575102690648907
f1 overall: 0.348


pp:  0.2
precicion_overall: 0.9989417989417989
recall_overall: 0.48391032976393444
f1 overall: 0.635


pp:  0.3
precicion_overall: 0.9939143489563657
recall_overall: 0.6879860171689008
f1 overall: 0.804


pp:  0.4
precicion_overall: 0.9853641328530759
recall_overall: 0.7933175685928892
f1 overall: 0.874


pp:  0.5
precicion_overall: 0.9890333152031516
recall_overall: 0.849719542337853
f1 overall: 0.911
    
    
    


#######################
# wordnet corpus 

vocabulary: 50000
sum_examples = 100
0.316
0.316
0.247
0.197
0.184
    
    
sum_examples = 500

pp:  0.1
precicion_overall: 0.6620126287541147
recall_overall: 0.2253642181118961
f1 overall: 0.309


pp:  0.2
precicion_overall: 0.4183446497049004
recall_overall: 0.48511057189407486
f1 overall: 0.432


pp:  0.3
precicion_overall: 0.3127062099596427
recall_overall: 0.6793736221434142
f1 overall: 0.409


pp:  0.4
precicion_overall: 0.26618831270029525
recall_overall: 0.7874246216705306
f1 overall: 0.377


pp:  0.5
precicion_overall: 0.23443659889114157
recall_overall: 0.8488645336677182
f1 overall: 0.35


sum_examples = 1000
0.310
0.498
0.497
0.472
0.445

#######################
# google corpus 

vocabulary: 50000
sum_examples = 100 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

.357
.267
.218
.201 
.175
    
sum_examples = 500 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
0.468
0.474
0.429
0.395
0.369
    
sum_examples = 1000
0.539
0.628
0.621
0.587
0.548


#######################
# closed cat corpus  13

vocabulary: 50000
    
sum_examples = 100 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
0.443
0.435
0.361
0.311
0.312

sum_examples = 500 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
0.582  
0.613
0.585
0.566
0.555

sum_examples = 1000    
0.610
0.652
0.660
0.636
0.629


SyntaxError: invalid syntax (<ipython-input-4-6174e7322fbd>, line 18)

In [ ]:
Experiments to be done:

ssn corpora with .. embeddings
--> closed_cat the best
closed_cat
analogy corpus
wordnet corpus

ssn embeddings models with closed_cat
--> ... the best
skipGram
glove
fasttext 

cls num negative samples with closed_cat
100
500
1000



'''
recall_scrore_average_no_noise = recall_scrore_average/itter
precision_scrore_average_no_noise = precision_scrore_average/itter
print('recall_scrore_average_no_noise: '+ str(recall_scrore_average_no_noise))
print('precision_scrore_average_no_noise: '+ str(precision_scrore_average_no_noise))
print('f1 with noise: '+ str(2*precision_scrore_average_no_noise*recall_scrore_average_no_noise/(recall_scrore_average_no_noise+precision_scrore_average_no_noise)))
print('')
'''

'''
recall_scrore_average_no_noise = (recall_scrore_average/itter*14-1)/13
precision_scrore_average_no_noise = (precision_scrore_average/itter*14-1)/13
f1_average_no_noise = 2*precision_scrore_average_no_noise*recall_scrore_average_no_noise/(recall_scrore_average_no_noise+precision_scrore_average_no_noise)

print('')
print('pp: ', pp)
print('recall_scrore_average_no_noise: '+ str(recall_scrore_average_no_noise))
print('precision_scrore_average_no_noise: '+ str(precision_scrore_average_no_noise))
print('f1 without noise: '+ str(f1_average_no_noise))
'''

